In [25]:
import ast
from typing import List
from os import listdir
import spotipy.util as util
import spotipy
import requests
import pandas as pd

In [3]:
def get_streamings(path: str = 'MyData') -> List[dict]:
    
    files = ['MyData/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings

In [5]:
streamed = get_streamings()

In [9]:
username = 'your_username_here'
client_id ='your_client_id'
client_secret = 'your_client_secret'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [20]:
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,}
    params = [('q', track_name),('type', 'track'),]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [22]:
lucy_id = get_id('Lucy', token)
print(lucy_id)

5RxpYHVbGJPOvSEATQyg9P


In [26]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [27]:
lucy_features = get_features(lucy_id, token)
print(lucy_features)

{'danceability': 0.647, 'energy': 0.537, 'key': 6, 'loudness': -12.006, 'mode': 0, 'speechiness': 0.21, 'acousticness': 0.43, 'instrumentalness': 0.000258, 'liveness': 0.119, 'valence': 0.846, 'tempo': 84.46, 'type': 'audio_features', 'id': '5RxpYHVbGJPOvSEATQyg9P', 'uri': 'spotify:track:5RxpYHVbGJPOvSEATQyg9P', 'track_href': 'https://api.spotify.com/v1/tracks/5RxpYHVbGJPOvSEATQyg9P', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5RxpYHVbGJPOvSEATQyg9P', 'duration_ms': 142500, 'time_signature': 4}


In [28]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName'] 
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})

In [29]:
import pandas as pd
df = pd.DataFrame(with_features)
df.to_csv('streaming_history.csv')